In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from src.env.DunderBotEnv import DunderBotEnv
import src.run as run
import src.load as load
import src.preprocess as preprocess
import src.data_raw as data_raw

from src.util.config import get_config
config = get_config()

%load_ext autoreload
%autoreload 2
%matplotlib inline

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

loading config from config/config.yml


# TODO:

- Add check for NaNs after head:ing with number of NaNs from config. Include infs. Abort if any. 
    * Check if there are any NaNs for 1m data.
- Consider ffill nan handling for TI:s (and likely dropping the filled data after)
- Consider this as two separate points for MLP and LSTM. Different requirements (see other point)
- Check/Enable/Add exploration noise (for PPO2, use the entropy flag), use https://stable-baselines.readthedocs.io/en/master/common/schedules.html. Will need to assess if model is still unstable.
- setup performance output, what we need to make data driven decision
    *e.g. https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/1_getting_started.ipynb#scrollTo=63M8mSKR-6Zt. Some Medium articles uses Sharpe values, investigate.
- Download and enable order book data (how do you aggregate orders in time bins?)
- Experiment with orders as features
    * e.g. shape of order book etc.
- self.stationarize_rewards?
- Set good period for ti features
- Test other TI packages for better efficiency
- Revisit technical features
- Test additional models to PPO2. Be careful though, many require different setting in which to run optimally.
- Hyperparameter search
- Explore different gamma (Discount factor)
- Correct for displaced index due to specified TI NaNs
- Checkout frame stacking as a poor man's lstm (unlikely that it's useful, since we're dealing with time series)
- Get the available LSTM networks working and compare with MLP (tips: https://github.com/hill-a/stable-baselines/issues/646). NOTE: LSTMs also need state during prediction as well as proper env counts for train and test (https://github.com/hill-a/stable-baselines/issues/166 and https://stable-baselines.readthedocs.io/en/master/guide/examples.html#recurrent-policies). Also: https://github.com/hill-a/stable-baselines/issues/308
- Understand more advanced custom policy networks better (need to understand terminology surronding it, e.g. environment, n_steps etc). See also discussion here (and elsewhere among Issues: https://github.com/hill-a/stable-baselines/issues/759, https://github.com/hill-a/stable-baselines/issues/278, https://github.com/hill-a/stable-baselines/issues/646)
- understand the rendering during learning (prob need to read the code to understand all callback fields)
- untangle index weirdness (basically, try to enable prediction the last timestep, and make running interface easier (and more robust))
    * Check info box for reason and solution: https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html
    * This might also help (at the end) https://github.com/hill-a/stable-baselines/issues/779
- Check float number type



# DONE: 
- add config
- modularize rewards
- own actions
- add more detailed actions for buy and sell amounts (still Discrete, I believe)
- wrap running code in function in main.py (to enable more dynamic data loading and train/test)
- own observations
- remove the bias when getting the observations (suggestion in the repo issues)
- download more interesting data and manually preprocess
- load interesting data, specified in config
- understand the time steps in learn and predict
- setup basic monitoring of training using TensorBoard
- Visualize basic performance
- Add support for fractions of shares
- Understand what should belong in what timestep (also decide on which price to plot as a result)
- add distributions of actions as output during prediction
- Refactor `_take_action` (might be something off in some calculations, also update terminology)
- Add commission and slippage
- Refactor current reward and add some new ones
- Add smallest amount of bitcoin we can own, as well as trading precision
- Add portfolio features
- Add reward output
- setup train/test functionality
- Enable saving/loading trained models
- Normalize observations
- Add env checker (disbled by default, but passed)
- lock down the seeds
- Add finer temporal resolution in data (minute granularity)
- Add support for higher frequency data: NaN handling and pushed starting timestamp
- Add technical indicator features
- Add benchmarks
- Add dynamic data download and preparation by downloading all transactions -> render OHLC in multiple time granularities
- Add full support for multiprocessing
- Suppress tensorflow warnings

# Run

In [3]:
%%time
data_raw.download_and_process()
df = load.load_OHLC_data()
df = preprocess.preprocess_data(df=df)
config.n_cpu=1
env = run.setup_env(df=df)

Processed data avilable locsally, no downloading and raw data processing needed.
LOAD: Loading data with 17575621 samples
PREPROCESS: Dropping 0 NaNs out of 17575621 samples (0.0%) from input file
PREPROCESS: Dropping unused data, 17575621 -> 14010 samples
PREPROCESS: Adding technical features...
Done.
Setting up environment using 1 cores...
Resetting to timesteps: start 17561685, end 17565620.
[4.03370000e+03 4.03373000e+03 4.03373000e+03 4.03372000e+03
 4.03372000e+03 4.03373000e+03 4.03372000e+03 4.03373000e+03
 4.03373000e+03 4.03373000e+03 4.56601000e-03 2.43056720e-01
 1.18430100e-02 8.63492120e-01 1.55485000e-03 3.55817970e-01
 8.02107600e-02 1.18429200e-02 6.62589000e-03 9.81046000e-02
 1.00000000e+04 1.00000000e+04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.41200291e-03 4.03371500e+03 4.03398500e+03
 4.03370500e+03 4.19915702e+01 2.78147834e-02 6.34887947e+01
 5.94231674e+01 1.87844481e+01 2.30159064e-17]
[4.03373000e+03 4.03373000e+03 4.03372000e+03 4.03372000e+03
 4.033

In [4]:
%%time
model = run.train(env=env, logging=True)

RUN: Training for 4000 serial timesteps and 4000 total timesteps...
Resetting to timesteps: start 17561685, end 17565620.
[4.03370000e+03 4.03373000e+03 4.03373000e+03 4.03372000e+03
 4.03372000e+03 4.03373000e+03 4.03372000e+03 4.03373000e+03
 4.03373000e+03 4.03373000e+03 4.56601000e-03 2.43056720e-01
 1.18430100e-02 8.63492120e-01 1.55485000e-03 3.55817970e-01
 8.02107600e-02 1.18429200e-02 6.62589000e-03 9.81046000e-02
 1.00000000e+04 1.00000000e+04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.41200291e-03 4.03371500e+03 4.03398500e+03
 4.03370500e+03 4.19915702e+01 2.78147834e-02 6.34887947e+01
 5.94231674e+01 1.87844481e+01 2.30159064e-17]
[4.03373000e+03 4.03373000e+03 4.03372000e+03 4.03372000e+03
 4.03373000e+03 4.03372000e+03 4.03373000e+03 4.03373000e+03
 4.03373000e+03 4.03373000e+03 2.43056720e-01 1.18430100e-02
 8.63492120e-01 1.55485000e-03 3.55817970e-01 8.02107600e-02
 1.18429200e-02 6.62589000e-03 9.81046000e-02 2.46676100e-02
 1.00000000e+04 1.00000000e+04 0.00000

[ 4.03500000e+03  4.03499000e+03  4.03500000e+03  4.03500000e+03
  4.03500000e+03  4.03499000e+03  4.03500000e+03  4.03374000e+03
  4.03464000e+03  4.03464000e+03  2.22185000e-03  2.49077500e-02
  7.11412000e-02  7.35134700e-02  4.85823700e-02  8.91199000e-03
  6.16743000e-03  2.37129100e-02  8.14022000e-03  1.23310200e-02
  5.73789000e+03  9.99999000e+03  1.05637723e+00  0.00000000e+00
  0.00000000e+00  2.84320231e-02  4.03437000e+03  4.03434500e+03
  4.03437000e+03  4.91279619e+01 -9.80855077e-02  3.36372885e+01
  5.09008955e+01  4.87197450e+01 -1.83312699e-16]
[ 4.03499000e+03  4.03500000e+03  4.03500000e+03  4.03500000e+03
  4.03499000e+03  4.03500000e+03  4.03374000e+03  4.03464000e+03
  4.03464000e+03  4.03464000e+03  2.49077500e-02  7.11412000e-02
  7.35134700e-02  4.85823700e-02  8.91199000e-03  6.16743000e-03
  2.37129100e-02  8.14022000e-03  1.23310200e-02  4.74325200e-02
  4.59031000e+03  9.99999000e+03  1.34080855e+00  0.00000000e+00
  0.00000000e+00  2.88674596e-02  4.0343

[4.03500000e+03 4.03499000e+03 4.03500000e+03 4.03500000e+03
 4.03511000e+03 4.03653000e+03 4.03653000e+03 4.03649000e+03
 4.03649000e+03 4.03650000e+03 8.14022000e-03 6.80978000e-03
 3.57814000e-03 2.79560000e-02 1.00100000e+00 1.87383800e-02
 1.00000000e-01 8.13800000e-02 5.28376000e-03 2.19684430e-01
 5.64868000e+03 1.00028900e+04 1.07871304e+00 0.00000000e+00
 0.00000000e+00 6.46075307e-02 4.03585500e+03 4.03523000e+03
 4.03585500e+03 8.70473871e+01 1.62064860e-01 3.52758935e+01
 1.18863163e+01 8.78807572e+01 2.00789721e-01]
[4.03499000e+03 4.03500000e+03 4.03500000e+03 4.03511000e+03
 4.03653000e+03 4.03653000e+03 4.03649000e+03 4.03649000e+03
 4.03650000e+03 4.03650000e+03 6.80978000e-03 3.57814000e-03
 2.79560000e-02 1.00100000e+00 1.87383800e-02 1.00000000e-01
 8.13800000e-02 5.28376000e-03 2.19684430e-01 3.33031000e-03
 6.51953000e+03 1.00029100e+04 8.62970430e-01 0.00000000e+00
 0.00000000e+00 6.82935024e-02 4.03585750e+03 4.03523000e+03
 4.03585500e+03 8.70473871e+01 1.46513

[ 4.03680000e+03  4.03521000e+03  4.03521000e+03  4.03520000e+03
  4.03409000e+03  4.03483000e+03  4.03451000e+03  4.03451000e+03
  4.03409000e+03  4.03350000e+03  4.50359200e-02  3.08357000e-03
  1.77496400e-02  5.05919127e+00  1.06014700e-02  2.39484000e-02
  5.79495000e-03  4.74258200e-02  3.33137000e-03  1.50000000e-01
  5.44235000e+03  9.99884000e+03  1.12949549e+00  0.00000000e+00
  0.00000000e+00  1.16058464e-01  4.03475250e+03  4.03515000e+03
  4.03515000e+03  2.20260340e+01 -2.96849053e-01  4.69038665e+01
  7.77069575e+01  2.22228559e+01 -7.07152561e-01]
[ 4.03521000e+03  4.03521000e+03  4.03520000e+03  4.03409000e+03
  4.03483000e+03  4.03451000e+03  4.03451000e+03  4.03409000e+03
  4.03350000e+03  4.03351000e+03  3.08357000e-03  1.77496400e-02
  5.05919127e+00  1.06014700e-02  2.39484000e-02  5.79495000e-03
  4.74258200e-02  3.33137000e-03  1.50000000e-01  1.23332300e-02
  4.35388000e+03  9.99817000e+03  1.39935293e+00  0.00000000e+00
  0.00000000e+00  1.23605220e-01  4.0347

[ 4.03432000e+03  4.03432000e+03  4.03352000e+03  4.03660000e+03
  4.03660000e+03  4.03361000e+03  4.03475000e+03  4.03356000e+03
  4.03350000e+03  4.03351000e+03  5.57476510e-01  2.42189200e-02
  3.19604250e+00  4.64269000e-03  7.11673000e-02  1.50747200e-02
  2.59270000e-03  2.37098500e-02  2.94675040e-01  2.46689000e-03
  3.20111000e+03  9.99867000e+03  1.68527598e+00  0.00000000e+00
  0.00000000e+00  8.38356999e-02  4.03505000e+03  4.03515000e+03
  4.03505000e+03  4.44310005e+01 -9.35674661e-02  2.73890730e+01
  5.55610177e+01  3.98578205e+01 -1.88294272e-01]
[ 4.03432000e+03  4.03352000e+03  4.03660000e+03  4.03660000e+03
  4.03361000e+03  4.03475000e+03  4.03356000e+03  4.03350000e+03
  4.03351000e+03  4.03351000e+03  2.42189200e-02  3.19604250e+00
  4.64269000e-03  7.11673000e-02  1.50747200e-02  2.59270000e-03
  2.37098500e-02  2.94675040e-01  2.46689000e-03  2.37213800e-02
  2.40083000e+03  9.99869000e+03  1.88368320e+00  0.00000000e+00
  0.00000000e+00  8.57216017e-02  4.0350

...

[ 3.99944000e+03  3.99944000e+03  3.99943000e+03  3.99885000e+03
  3.99884000e+03  3.99885000e+03  3.99885000e+03  3.99885000e+03
  3.99885000e+03  3.99877000e+03  1.54275300e-02  3.58856200e-02
  1.33284000e-01  8.00000000e+00  1.11143530e-01  1.70586400e-02
  8.37356000e-02  1.00000000e+00  1.35469159e+00  1.67942746e+00
  8.41390000e+02  9.95639000e+03  2.27940508e+00  0.00000000e+00
  0.00000000e+00  2.86259391e-02  3.99769250e+03  3.98621000e+03
  3.99629500e+03  6.72590922e+01 -5.92181554e-01  5.67553398e+01
  2.40172386e+01  6.22512308e+01  6.27056693e-01]
[ 3.99944000e+03  3.99943000e+03  3.99885000e+03  3.99884000e+03
  3.99885000e+03  3.99885000e+03  3.99885000e+03  3.99885000e+03
  3.99877000e+03  3.99797000e+03  3.58856200e-02  1.33284000e-01
  8.00000000e+00  1.11143530e-01  1.70586400e-02  8.37356000e-02
  1.00000000e+00  1.35469159e+00  1.67942746e+00  2.00297000e+00
  4.20700000e+02  9.95621000e+03  2.38461118e+00  0.00000000e+00
  1.00000000e+00  3.91813373e-02  3.9975

[ 3.99502000e+03  3.99502000e+03  3.99502000e+03  3.99807000e+03
  3.99807000e+03  3.99947000e+03  3.99947000e+03  3.99762000e+03
  3.99914000e+03  3.99753000e+03  9.39256050e-01  5.32300000e-01
  1.61300000e-01  3.59253200e-02  1.22000000e-01  2.39222000e-02
  2.48819000e-03  7.01747400e-02  5.84929000e-03  2.40000000e-03
  5.53303000e+03  9.95279000e+03  1.10517835e+00  0.00000000e+00
  0.00000000e+00  1.90878725e-01  3.99704500e+03  3.99631000e+03
  3.99684500e+03  5.43815931e+01  3.67078758e-01  3.44769359e+01
  4.49345387e+01  4.18513952e+01 -3.00592128e-01]
[ 3.99502000e+03  3.99502000e+03  3.99807000e+03  3.99807000e+03
  3.99947000e+03  3.99947000e+03  3.99762000e+03  3.99914000e+03
  3.99753000e+03  3.99625000e+03  5.32300000e-01  1.61300000e-01
  3.59253200e-02  1.22000000e-01  2.39222000e-02  2.48819000e-03
  7.01747400e-02  5.84929000e-03  2.40000000e-03  3.58881400e-02
  5.53303000e+03  9.95101000e+03  1.10517835e+00  0.00000000e+00
  0.00000000e+00  1.89598740e-01  3.9970

[4.01500000e+03 4.01545000e+03 4.01849000e+03 4.02044000e+03
 4.02199000e+03 4.02230000e+03 4.02245000e+03 4.02350000e+03
 4.02946000e+03 4.03229000e+03 1.29616200e-02 6.35800000e-02
 1.23262570e+01 1.52908179e+01 3.17270513e+00 2.10000000e-01
 1.00176000e+00 2.72800000e-02 3.74908290e+00 5.99069265e+00
 9.01500000e+02 1.00022400e+04 2.25855006e+00 1.00000000e+00
 0.00000000e+00 7.94549883e-01 4.01922500e+03 4.01325500e+03
 4.01480500e+03 9.64234515e+01 1.34685035e+00 6.56420805e+01
 5.17341503e+00 9.27977718e+01 9.61963014e-01]
[4.01545000e+03 4.01849000e+03 4.02044000e+03 4.02199000e+03
 4.02230000e+03 4.02245000e+03 4.02350000e+03 4.02946000e+03
 4.03229000e+03 4.03392000e+03 6.35800000e-02 1.23262570e+01
 1.52908179e+01 3.17270513e+00 2.10000000e-01 1.00176000e+00
 2.72800000e-02 3.74908290e+00 5.99069265e+00 2.70958129e+01
 3.17828000e+03 1.00086300e+04 1.69391255e+00 0.00000000e+00
 0.00000000e+00 8.52178766e-01 4.02346000e+03 4.01699500e+03
 4.01931000e+03 9.66968277e+01 1.45942

[ 4.03068000e+03  4.01768000e+03  4.02054000e+03  4.02054000e+03
  4.02054000e+03  4.02285000e+03  4.02431000e+03  4.02657000e+03
  4.02819000e+03  4.02777000e+03  1.90931000e+01  4.52326617e+01
  1.52209514e+01  5.06486341e+00  2.28364363e+00  1.16979270e+01
  1.90200000e+00  1.00892329e+00  1.72628000e+00  1.23520600e-02
  2.01385000e+03  1.00002400e+04  1.98262409e+00  0.00000000e+00
  0.00000000e+00  9.19686523e-01  4.02867750e+03  4.02653000e+03
  4.03335500e+03  4.67945202e+01 -9.91782400e-01  4.19687730e+01
  3.41950294e+01  3.29438504e+01 -2.63833403e-01]
[ 4.01768000e+03  4.02054000e+03  4.02054000e+03  4.02054000e+03
  4.02285000e+03  4.02431000e+03  4.02657000e+03  4.02819000e+03
  4.02777000e+03  4.02777000e+03  4.52326617e+01  1.52209514e+01
  5.06486341e+00  2.28364363e+00  1.16979270e+01  1.90200000e+00
  1.00892329e+00  1.72628000e+00  1.23520600e-02  6.91962000e-03
  1.34257000e+03  9.99941000e+03  2.14928786e+00  0.00000000e+00
  0.00000000e+00  8.65810816e-01  4.0286

KeyboardInterrupt: 

In [4]:
%%time
run.predict(df=df, rendermode='human')

AssertionError: Number of predict timesteps requested larger than in config (10000 > 9989)

In [6]:
import pickle
with open('all_dict_pred.pickle', 'rb') as f:
    x = pickle.load(f)
x

{'current_step': 18,
 'net_worths': [10000.0,
  10000.0,
  10000.0,
  10001.7,
  9993.31,
  9845.08,
  9818.83,
  9857.64,
  9811.18,
  9826.9,
  9849.13,
  9841.65,
  9846.6,
  9906.72,
  9857.82,
  9825.55,
  9844.28,
  9835.73,
  9843.79,
  9853.91,
  9847.28,
  9858.84,
  9884.08,
  9846.62,
  9846.11,
  9859.04,
  9865.74,
  9849.82,
  9871.96,
  9886.04,
  9852.41,
  9828.36,
  9781.67,
  9796.26,
  9789.61,
  9787.16,
  9798.66,
  9821.71,
  9819.04,
  9825.21,
  9813.02,
  9817.28,
  9809.9,
  9810.35,
  9815.73,
  9852.16,
  9844.47,
  9845.15,
  9833.28,
  9836.58,
  9863.32,
  9843.26,
  9846.9,
  9838.6,
  9843.73,
  9840.44,
  9971.54,
  9981.18,
  9980.54,
  9981.68,
  9980.22,
  9984.06,
  10005.04,
  9992.58,
  9998.53,
  10012.4,
  10006.93,
  9995.75,
  9951.78,
  9922.8,
  9824.37,
  9840.96,
  9803.94,
  9764.1,
  9780.03,
  9798.29,
  9787.19,
  9793.24,
  9805.33,
  9789.97,
  9796.14,
  9795.5,
  9814.97,
  9812.16,
  9803.83,
  9817.89,
  9804.12,
  9807.79,
  9

In [8]:
x.keys()

dict_keys(['current_step', 'net_worths', 'trades', 'account_history', 'rewards'])